In [2]:
import numpy as np # Do simulation in numpy to avoid accidentally carrying gradients



In [ ]:
# A CA rule which takes 'benzene' and registers it as a transistor
# A rule which takes square and removes them. Maybe remove a random part of them?
# Rule which states a wire must have a specifc shape.
# Rule that a wire mustn't be lonely.

In [9]:
sampleImg = np.array(
    [
        [0, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
    ]
)
sampleImg.shape

(8, 8)

In [ ]:


transistorKernel = np.array(
    [
        [1, 1, 1],
        [1, 0, 1],
        [1, 1, 1],
    ]
)

nullKernel = np.array(
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    ]
)

anyKernel =  np.array(
    [
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
    ]
)


def AnyRule(kernel, region):
    return np.any(region * kernel)

def MatchRule(kernel, region):
    return np.array_equal(region, kernel)

def ApplyCA(input, kernel, rule):
    padddedInput = np.pad(input, 1, mode='constant')

    outputArray = np.zeros_like(input)

    for i in range(input.shape[0]):
        for j in range(input.shape[1]):
            region = padddedInput[i:i+3, j:j+3]
            outputArray[i][j] = rule(region, kernel)

    return outputArray

ApplyCA(ApplyCA(sampleImg, transistorKernel, MatchRule), anyKernel, AnyRule)

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [32]:
transitorImg = ApplyCA(ApplyCA(sampleImg, transistorKernel, MatchRule), anyKernel, AnyRule)
wireImg = (sampleImg - transitorImg).clip(0, 1)
wireImg

array([[0, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [55]:
# I want to test
# 2 inputs on left,
# 1 output on right,
# 1 transistor generated.

# Wires can be calculated as a disjoint set, such that connections only require one calculation.

def WireSets(WireImage):
    connections = np.full_like(WireImage, None, dtype=object) 
    WireImage = np.pad(WireImage, pad_width=((1, 0), (1, 0)), mode='constant') # This is only padded to avoid index out of bounds errors.

    def TraceSetEnd(position, newParent):
        # Traces the connections until it reaches an element pointing to itself (the end)
        # Each value iterated over in the process have their parent set to the end value.
        
        while connections[position] is not newParent:
            nextPosition = connections[position]
            connections[position] = newParent
            position = nextPosition

    # Create disjoint sets of connected wires
    for i in range(connections.shape[0]):
        for j in range(connections.shape[1]):
            if WireImage[i + 1, j + 1] == 1: # If current pixel has a wire
                # Check left and up for connections
                
                if WireImage[i, j + 1] == 1: # and the pixel to the left has a wire
                    connections[i, j] = connections[i - 1, j]
                    if WireImage[i + 1, j] == 1: # if both left and up have a wire, set up to use left as parent.
                        TraceSetEnd((i, j - 1), connections[i, j])
                elif WireImage[i + 1, j] == 1: # or the pixel above has a wire
                    connections[i, j] = connections[i, j - 1]
                else: # If no connections found, set as set containing self
                    connections[i, j] = (i, j)

    def TraceSet(position):
        # This should be changed to set values during travel as a dynamic programming approach.
        while position is not connections[position]:
            position = connections[position]
        
        return position

    connectionDict = {}
    setCount = 0
    connectionMap = np.zeros_like(connections)
    for i in range(connections.shape[0]):
        for j in range(connections.shape[1]):
            if connections[i, j] is not None:
                setRepressentative = TraceSet(connections[i, j])

                if setRepressentative not in connectionDict:
                    setCount += 1
                    connectionDict[setRepressentative] = setCount

                connectionMap[i, j] = connectionDict[setRepressentative]

    return connectionMap 

WireSets(wireImg)

array([[0, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2],
       [0, 0, 0, 0, 0, 0, 0, 2],
       [3, 3, 3, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [53]:
for position in testOut:
    print(position)

[None (0, 1) (0, 1) (0, 1) (0, 1) (0, 1) None None]
[(0, 1) (0, 1) (0, 1) None None (0, 1) None None]
[None None None None None (0, 1) None None]
[None None None None None None None (3, 7)]
[None None None None None None None (3, 7)]
[(5, 0) (5, 0) (5, 0) (5, 0) None None None None]
[None None None None None None None None]
[None None None None None None None None]


In [ ]:
# Ok, now i have a good model for the circuit simulation.
# How should i simulate it?
# I think the transistor map isn't that useful, i should change it to just return a list of transistor positions.
#  No, wait. It is useful, if only for removing wires.
# Because what i want is
# Map -> list of components and connections -> output

In [ ]:
# For each component socket
# Query the connection map for what it is connected to

# This has to result in a new dictionary
# wireSet -> connectedCompoenents
# This set can then be translated to
# connectedCompoennets -> per Component socket, other sockets that can be reached.

In [ ]:
# Perhaps model transistors as having single pixel sockets

#These are in y, x coordinates. That is annoying. What did i do.
outSockets = [(0,1), (5,0), (4, 7)]
inSockets = [(4, 3), (3, 7), (2,5), (6, 5)]
